# Trabajo Práctico 2 - Clasificación aplicada a la EPH 

## Gil Deza, Hüppi Lo Prete, Walker 

### Parte 1 - Analizando la base 

In [2]:
# Importamos paquetes a utilizar
import pandas as pd
import matplotlib as plt

### Punto 2

In [4]:
# Importamos la base
df=pd.read_excel("usu_individual_T122.xls")
df


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,2,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
1,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,3,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
2,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,4,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
3,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,1,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
4,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,2,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49701,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,2,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49702,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,1,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777
49703,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,2,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777
49704,TQRMNOQRPHMMLPCDEFIAH00707721,2022,1,1,1,1,43,S,2,678,...,NaN,6,34000.0,5,5.0,5,5.0,NaN,5,2274


In [5]:
# Armamos un nuevo df solo con las observaciones de Buenos Aires y Gran Buenos Aires
df_ba = df.loc[df['AGLOMERADO'].isin([32, 33])]
df_ba


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
3,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,1,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
4,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,2,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
5,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,3,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
75,TQRMNOPQQHKMRLCDEIJAH00780169,2022,1,1,1,1,1,S,33,456,...,NaN,0,0.0,0,NaN,0,0.0,NaN,0,647
76,TQRMNOSXRHJMTRCDEIJAH00693084,2022,1,1,1,1,1,S,33,1199,...,NaN,1,10000.0,1,NaN,1,1.0,NaN,1,1470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49699,TQRMNOTTSHKNLSCDEIIAD00780102,2022,1,1,2,1,1,S,32,2839,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
49700,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,1,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49701,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,2,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49702,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,1,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777


In [22]:
#Hacemos print de las observaciones que tengan valores de ingresos totales familiares (ITF) y per cápita (IPCF) negativos.

print(df_ba[df_ba["ITF"] < 0])
#Obtenemos un dataframe vacío, por lo que no hay observaciones que cumplan con esas características.

print(df_ba[df_ba["IPCF"] < 0])
#Obtenemos un dataframe vacío, por lo que no hay observaciones que cumplan con esas características.

print(df_ba[df_ba["P21"] < 0])
#Obtenemos un dataframe con datos de ingreso negativos, por lo que procedemos a eliminarlos.

print(df_ba[df_ba["P47T"] < 0])
#Obtenemos un dataframe con datos de ingreso negativos, por lo que procedemos a eliminarlos.

print(df_ba[df_ba["CH06"] < 0])
#Obtenemos un dataframe con datos de edad negativos, por lo que procedemos a eliminarlos.

Empty DataFrame
Columns: [CODUSU, ANO4, TRIMESTRE, NRO_HOGAR, COMPONENTE, H15, REGION, MAS_500, AGLOMERADO, PONDERA, CH03, CH04, CH05, CH06, CH07, CH08, CH09, CH10, CH11, CH12, CH13, CH14, CH15, CH15_COD, CH16, CH16_COD, NIVEL_ED, ESTADO, CAT_OCUP, CAT_INAC, IMPUTA, PP02C1, PP02C2, PP02C3, PP02C4, PP02C5, PP02C6, PP02C7, PP02C8, PP02E, PP02H, PP02I, PP03C, PP03D, PP3E_TOT, PP3F_TOT, PP03G, PP03H, PP03I, PP03J, INTENSI, PP04A, PP04B_COD, PP04B1, PP04B2, PP04B3_MES, PP04B3_ANO, PP04B3_DIA, PP04C, PP04C99, PP04D_COD, PP04G, PP05B2_MES, PP05B2_ANO, PP05B2_DIA, PP05C_1, PP05C_2, PP05C_3, PP05E, PP05F, PP05H, PP06A, PP06C, PP06D, PP06E, PP06H, PP07A, PP07C, PP07D, PP07E, PP07F1, PP07F2, PP07F3, PP07F4, PP07F5, PP07G1, PP07G2, PP07G3, PP07G4, PP07G_59, PP07H, PP07I, PP07J, PP07K, PP08D1, PP08D4, PP08F1, PP08F2, PP08J1, PP08J2, ...]
Index: []

[0 rows x 177 columns]
Empty DataFrame
Columns: [CODUSU, ANO4, TRIMESTRE, NRO_HOGAR, COMPONENTE, H15, REGION, MAS_500, AGLOMERADO, PONDERA, CH03, CH04, 

In [23]:
#Eliminamos las observaciones con edades negativas.
df_ba_limpio = df_ba[df_ba.CH06>=0]

#Eliminamos las observaciones con ingreso negativo.
df_ba_limpio = df_ba_limpio[df_ba_limpio.P21>=0]

#Eliminamos las observaciones con ingreso negativo.
df_ba_limpio = df_ba_limpio[df_ba_limpio.P47T>=0]

#Chequeamos que al imprimir el dataframe seleccionando edades negativas devuelva un dataframe vacío.
print(df_ba_limpio[df_ba_limpio["CH06"] < 0])

#Chequeamos que al imprimir el dataframe seleccionando ingresos negativos devuelva un dataframe vacío.
print(df_ba_limpio[df_ba_limpio["P21"] < 0])

#Chequeamos que al imprimir el dataframe seleccionando ingresos negativos devuelva un dataframe vacío.
print(df_ba_limpio[df_ba_limpio["P47T"] < 0])

Empty DataFrame
Columns: [CODUSU, ANO4, TRIMESTRE, NRO_HOGAR, COMPONENTE, H15, REGION, MAS_500, AGLOMERADO, PONDERA, CH03, CH04, CH05, CH06, CH07, CH08, CH09, CH10, CH11, CH12, CH13, CH14, CH15, CH15_COD, CH16, CH16_COD, NIVEL_ED, ESTADO, CAT_OCUP, CAT_INAC, IMPUTA, PP02C1, PP02C2, PP02C3, PP02C4, PP02C5, PP02C6, PP02C7, PP02C8, PP02E, PP02H, PP02I, PP03C, PP03D, PP3E_TOT, PP3F_TOT, PP03G, PP03H, PP03I, PP03J, INTENSI, PP04A, PP04B_COD, PP04B1, PP04B2, PP04B3_MES, PP04B3_ANO, PP04B3_DIA, PP04C, PP04C99, PP04D_COD, PP04G, PP05B2_MES, PP05B2_ANO, PP05B2_DIA, PP05C_1, PP05C_2, PP05C_3, PP05E, PP05F, PP05H, PP06A, PP06C, PP06D, PP06E, PP06H, PP07A, PP07C, PP07D, PP07E, PP07F1, PP07F2, PP07F3, PP07F4, PP07F5, PP07G1, PP07G2, PP07G3, PP07G4, PP07G_59, PP07H, PP07I, PP07J, PP07K, PP08D1, PP08D4, PP08F1, PP08F2, PP08J1, PP08J2, ...]
Index: []

[0 rows x 177 columns]
Empty DataFrame
Columns: [CODUSU, ANO4, TRIMESTRE, NRO_HOGAR, COMPONENTE, H15, REGION, MAS_500, AGLOMERADO, PONDERA, CH03, CH04, 